# LeNet Lab Solution
![LeNet Architecture](lenet.png)
Source: Yan LeCun

## Load Data

Load the MNIST data, which comes pre-loaded with TensorFlow.

You do not need to modify this section.

In [1]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", reshape=False)
X_train, y_train           = mnist.train.images, mnist.train.labels
X_validation, y_validation = mnist.validation.images, mnist.validation.labels
X_test, y_test             = mnist.test.images, mnist.test.labels

assert(len(X_train) == len(y_train))
assert(len(X_validation) == len(y_validation))
assert(len(X_test) == len(y_test))

print()
print("Image Shape: {}".format(X_train[0].shape))
print()
print("Training Set:   {} samples".format(len(X_train)))
print("Validation Set: {} samples".format(len(X_validation)))
print("Test Set:       {} samples".format(len(X_test)))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.

Image Shape: (28, 28, 1)

Training Set:   55000 samples
Validation Set: 5000 samples
Test Set:       10000 samples


In [2]:
import numpy as np

# Pad images with 0s
X_train      = np.pad(X_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_validation = np.pad(X_validation, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_test       = np.pad(X_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')
    
print("Updated Image Shape: {}".format(X_train[0].shape))

Updated Image Shape: (32, 32, 1)


In [9]:
from sklearn.utils import shuffle

X_train, y_train = shuffle(X_train, y_train)

In [3]:
import tensorflow as tf

EPOCHS = 10
BATCH_SIZE = 128

## SOLUTION: Implement LeNet-5
Implement the [LeNet-5](http://yann.lecun.com/exdb/lenet/) neural network architecture.

This is the only cell you need to edit.
### Input
The LeNet architecture accepts a 32x32xC image as input, where C is the number of color channels. Since MNIST images are grayscale, C is 1 in this case.

### Architecture
**Layer 1: Convolutional.** The output shape should be 28x28x6.

**Activation.** Your choice of activation function.

**Pooling.** The output shape should be 14x14x6.

**Layer 2: Convolutional.** The output shape should be 10x10x16.

**Activation.** Your choice of activation function.

**Pooling.** The output shape should be 5x5x16.

**Flatten.** Flatten the output shape of the final pooling layer such that it's 1D instead of 3D. The easiest way to do is by using `tf.contrib.layers.flatten`, which is already imported for you.

**Layer 3: Fully Connected.** This should have 120 outputs.

**Activation.** Your choice of activation function.

**Layer 4: Fully Connected.** This should have 84 outputs.

**Activation.** Your choice of activation function.

**Layer 5: Fully Connected (Logits).** This should have 10 outputs.

### Output
Return the result of the 2nd fully connected layer.

In [4]:
from tensorflow.contrib.layers import flatten

def LeNet(x):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    # SOLUTION: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 1, 6), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(6))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b

    # SOLUTION: Activation.
    conv1 = tf.nn.relu(conv1)

    # SOLUTION: Pooling. Input = 28x28x6. Output = 14x14x6.
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # SOLUTION: Layer 2: Convolutional. Output = 10x10x16.
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(16))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    
    # SOLUTION: Activation.
    conv2 = tf.nn.relu(conv2)

    # SOLUTION: Pooling. Input = 10x10x16. Output = 5x5x16.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # SOLUTION: Flatten. Input = 5x5x16. Output = 400.
    fc0   = flatten(conv2)
    
    # SOLUTION: Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(400, 120), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    
    # SOLUTION: Activation.
    fc1    = tf.nn.relu(fc1)

    # SOLUTION: Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(84))
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    
    # SOLUTION: Activation.
    fc2    = tf.nn.relu(fc2)

    # SOLUTION: Layer 5: Fully Connected. Input = 84. Output = 10.
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(84, 10), mean = mu, stddev = sigma))
    fc3_b  = tf.Variable(tf.zeros(10))
    logits = tf.matmul(fc2, fc3_W) + fc3_b
    
    return logits

In [5]:
x = tf.placeholder(tf.float32, (None, 32, 32, 1))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 10)
rate = 0.001

In [6]:


#logits = LeNet(x)


# Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
mu = 0
sigma = 0.1

# SOLUTION: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 1, 6), mean = mu, stddev = sigma))
conv1_b = tf.Variable(tf.zeros(6))
conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b

# SOLUTION: Activation.
conv1 = tf.nn.relu(conv1)

# SOLUTION: Pooling. Input = 28x28x6. Output = 14x14x6.
conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

# SOLUTION: Layer 2: Convolutional. Output = 10x10x16.
conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean = mu, stddev = sigma))
conv2_b = tf.Variable(tf.zeros(16))
conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b

# SOLUTION: Activation.
conv2 = tf.nn.relu(conv2)

# SOLUTION: Pooling. Input = 10x10x16. Output = 5x5x16.
conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

# SOLUTION: Flatten. Input = 5x5x16. Output = 400.
fc0   = flatten(conv2)

# SOLUTION: Layer 3: Fully Connected. Input = 400. Output = 120.
fc1_W = tf.Variable(tf.truncated_normal(shape=(400, 120), mean = mu, stddev = sigma))
fc1_b = tf.Variable(tf.zeros(120))
fc1   = tf.matmul(fc0, fc1_W) + fc1_b

# SOLUTION: Activation.
fc1    = tf.nn.relu(fc1)

# SOLUTION: Layer 4: Fully Connected. Input = 120. Output = 84.
fc2_W  = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = mu, stddev = sigma))
fc2_b  = tf.Variable(tf.zeros(84))
fc2    = tf.matmul(fc1, fc2_W) + fc2_b

# SOLUTION: Activation.
fc2    = tf.nn.relu(fc2)

# SOLUTION: Layer 5: Fully Connected. Input = 84. Output = 10.
fc3_W  = tf.Variable(tf.truncated_normal(shape=(84, 10), mean = mu, stddev = sigma))
fc3_b  = tf.Variable(tf.zeros(10))
logits = tf.matmul(fc2, fc3_W) + fc3_b




cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [7]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [10]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
            
        validation_accuracy = evaluate(X_validation, y_validation)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './lenet')
    print("Model saved")
    
    # print weight and  bias
    
    

Training...

EPOCH 1 ...
Validation Accuracy = 0.969

EPOCH 2 ...
Validation Accuracy = 0.979

EPOCH 3 ...
Validation Accuracy = 0.982

EPOCH 4 ...
Validation Accuracy = 0.988

EPOCH 5 ...
Validation Accuracy = 0.985

EPOCH 6 ...
Validation Accuracy = 0.988

EPOCH 7 ...
Validation Accuracy = 0.986

EPOCH 8 ...
Validation Accuracy = 0.990

EPOCH 9 ...
Validation Accuracy = 0.988

EPOCH 10 ...
Validation Accuracy = 0.987

Model saved


In [11]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

INFO:tensorflow:Restoring parameters from .\lenet
Test Accuracy = 0.989


In [15]:
np.set_printoptions(threshold = 1e6)

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    #print("logits", sess.run(logits, feed_dict={x: X_test[0:1]}))
   

    flog = open("weight1.log", 'w')
    
    weight1 = sess.run(conv1_W)
    print("weight1: ", weight1.shape) 
    #print("weight1: ", weight1) 
    
    for i in range(5):
        for j in range(5):
            for o in range(6):
                if (weight1[i][j][0][o]*32768).astype(int) < 0 :
                    print("-", file = flog, end='')
                #print("16'd%d, "%(weight1[i][j][0][o]*256).astype(int) , file = flog, end=' ')
                print("16'd%d, "%abs((weight1[i][j][0][o]*32768).astype(int)) , file = flog, end=' ')
            print("", file = flog)
        print("", file = flog)
        
    print("==========================conv1_bias==========================", file = flog)        
    bias1 = sess.run(conv1_b)
    print("", file = flog)   
    for o in range(6):
        if (bias1[o]*32768*256).astype(int) < 0 :
            print("-", file = flog, end='')
        print("24'd%d, "%abs((bias1[o]*32768*256).astype(int)), file = flog, end=' ')
        
    flog.close()
    

####################################################    

    flog = open("weight2.log", 'w')
    
    weight2 = sess.run(conv2_W)
    print("weight2: ", weight2.shape)   
    
    for row in range(5):
        for col in range(5):
            for o in range(16):
                for i in range(6):
                    if (weight2[row][col][i][o]*32768).astype(int) < 0 :
                        print("-", file = flog, end='')
                    print("16'd%d, "%abs((weight2[row][col][i][o]*32768).astype(int)) , file = flog, end=' ')
                print("", file = flog)
            print("", file = flog)
        print("", file = flog)
    
    print("", file = flog)  
    
    bias2 = sess.run(conv2_b)
    print("bias2: ", bias2.shape)
    print("", file = flog)
    print("", file = flog)   
    print("==========================conv2_bias==========================", file = flog) 
    for o in range(16):
        if (bias2[o]*32768*256).astype(int) < 0 :
            print("-", file = flog, end='')
        print("24'd%d, "%abs((bias2[o]*32768*256).astype(int)), file = flog, end=' ')
        
        
    flog.close()

####################################################   
    flog = open("fc1.log", 'w')
    weight_fc1  = sess.run(fc1_W)
    
    print("weight_fc1: ", weight_fc1.shape)
    
    for o in range(120):
        for i in range(400):
            #print( "%2f, "%weight_fc1[i][o], file = flog, end=' ')
            #print("%6d, "%(weight_fc1[i][o]).astype(int) , file = flog, end=' ')
            if (weight_fc1[i][o]*32768).astype(int) < 0 :
                print("-", file = flog, end='')
            print("16'd%d, "%abs((weight_fc1[i][o]*32768).astype(int)) , file = flog, end=' ')
            if i%16==15 :
                print("", file = flog) 
        print("", file = flog)  
    bias_fc1  = sess.run(fc1_b)
    print("bias_fc1: ", bias_fc1.shape)
    print("==========================bias_fc1==========================", file = flog)   
    for o in range(120):
        if (bias_fc1[o]*32768*256).astype(int) < 0 :
            print("-", file = flog, end='')
        print("24'd%d, "%abs((bias_fc1[o]*32768*256).astype(int)) , file = flog, end=' ')
        if o%16==15 :
            print("", file = flog)  
    print("", file = flog)   
    
    fc1_re = sess.run(fc1, feed_dict={x: X_test[0:1]})
    print("fc1_re: ", fc1_re.shape)
    print("==========================fc1_re==========================", file = flog)   
    
    for i in range(120):
        print("%6d, "%(fc1_re[0][i]*256).astype(int), file = flog, end=' ')
        if i%16==15 :
            print("", file = flog) 
    print("", file = flog)  
    
    flog.close()
#################################################### 
    flog = open("fc2.log", 'w')
    weight_fc2  = sess.run(fc2_W)
    
    print("weight_fc2: ", weight_fc2.shape)
    
    for o in range(84):
        for i in range(120):
            if (weight_fc2[i][o]*32768).astype(int) < 0 :
                print("-", file = flog, end='')
            #print( "%2f, "%weight_fc1[i][o], file = flog, end=' ')
            #print("%6d, "%(weight_fc1[i][o]).astype(int) , file = flog, end=' ')
            print("16'd%d, "%abs((weight_fc2[i][o]*32768).astype(int)) , file = flog, end=' ')
            if i%16==15 :
                print("", file = flog) 
        print("", file = flog)  
    bias_fc2  = sess.run(fc2_b)
    print("bias_fc2: ", bias_fc2.shape)
    print("==========================bias_fc2==========================", file = flog)  
    for o in range(84):
        if (bias_fc2[o]*32768*256).astype(int) < 0 :
            print("-", file = flog, end='')
        print("24'd%d, "%abs((bias_fc2[o]*32768*256).astype(int)) , file = flog, end=' ')
        if o%16==15 :
            print("", file = flog)  
    print("", file = flog)   
    
    fc2_re = sess.run(fc2, feed_dict={x: X_test[0:1]})
    print("fc2_re: ", fc2_re.shape)
    print("==========================fc2_re==========================", file = flog)   
    
    for i in range(84):
        print("%6d, "%(fc2_re[0][i]*256).astype(int), file = flog, end=' ')
        if i%16==15 :
            print("", file = flog) 
    print("", file = flog)  
    
    flog.close()
#################################################### 
    flog = open("fc3.log", 'w')
    weight_fc3  = sess.run(fc3_W)
    
    print("weight_fc3: ", weight_fc3.shape)
    
    for o in range(10):
        for i in range(84):
            if (weight_fc3[i][o]*32768).astype(int) < 0 :
                print("-", file = flog, end='')
            #print( "%2f, "%weight_fc1[i][o], file = flog, end=' ')
            #print("%6d, "%(weight_fc1[i][o]).astype(int) , file = flog, end=' ')
            print("16'd%d, "%abs((weight_fc3[i][o]*32768).astype(int)) , file = flog, end=' ')
            if i%16==15 :
                print("", file = flog) 
        print("", file = flog)  
    bias_fc3  = sess.run(fc3_b)
    print("bias_fc3: ", bias_fc3.shape)
    print("==========================bias_fc3==========================", file = flog) 
    for o in range(10):
        if (bias_fc3[o]*32768*256).astype(int) < 0 :
            print("-", file = flog, end='')
        print("24'd%d, "%abs((bias_fc3[o]*32768*256).astype(int)) , file = flog, end=' ')
        if o%16==15 :
            print("", file = flog)  
    print("", file = flog)   
    
    fc3_re = sess.run(logits, feed_dict={x: X_test[0:1]})
    print("fc3_re: ", fc3_re.shape)
    print("==========================fc3_re==========================", file = flog)   
    
    for i in range(10):
        print("%6d, "%(fc3_re[0][i]*256).astype(int), file = flog, end=' ')
        if i%16==15 :
            print("", file = flog) 
    print("", file = flog)  
    
    flog.close()

INFO:tensorflow:Restoring parameters from .\lenet
weight1:  (5, 5, 1, 6)
weight2:  (5, 5, 6, 16)
bias2:  (16,)
weight_fc1:  (400, 120)
bias_fc1:  (120,)
fc1_re:  (1, 120)
weight_fc2:  (120, 84)
bias_fc2:  (84,)
fc2_re:  (1, 84)
weight_fc3:  (84, 10)
bias_fc3:  (10,)
fc3_re:  (1, 10)


In [14]:
# test 
np.set_printoptions(threshold = 1e6)

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    
        
####################################################     
    flog = open("conv1.log", 'w')
    
    conv1_test   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b
    relu1 = tf.nn.relu(conv1_test)
    
    conv_re = sess.run(conv1_test, feed_dict={x: X_test[0:1]})
    relu1_re = sess.run(relu1, feed_dict={x: X_test[0:1]})
    conv1
    pool1_re = sess.run(conv1, feed_dict={x: X_test[0:1]})
    print("conv1: ", conv_re.shape) 
    #print("conv1: ", conv_re) 
    print("relu1: ", relu1.shape) 
    print("pool1_re: ", pool1_re.shape) 
    
    
    print("conv1: ", conv_re.shape, file = flog) 
    #print("conv1: ", conv_re, file = flog) 
    for o in range(6):
        for i in range(28):
            for j in range(28):
                print(conv_re[0][i][j][o], file = flog, end=' ')
                #print(relu1_re[0][i][j][o], file = flog, end='     ')
            print("", file = flog)
        print("", file = flog)
    

    print("", file = flog)      
    for i in range(28):
        for j in range(28):
            for o in range(6):
                print("%6d, "%(conv_re[0][i][j][o]*256).astype(int), file = flog, end=' ')
            print("", file = flog)
        print("", file = flog)    
        
    print("==========================pool1_re==========================", file = flog)     
    
    print("", file = flog)      
    for i in range(14):
        for j in range(14):
            for o in range(6):
                print("%6d, "%(pool1_re[0][i][j][o]*256).astype(int), file = flog, end=' ')
            print("", file = flog)
        print("", file = flog)    
    flog.close()
    
####################################################        
    
    conv2_in = tf.placeholder(tf.float32, (None, 14, 14, 6))
    
    ones32x32x6 = np.ones(14*14*6)
    ones32x32x6 = ones32x32x6.reshape(1, 14,14,6)
    #tensor32x32x6
    
    flog = open("conv2.log", 'w')
    
    conv2_test   = tf.nn.conv2d(conv2_in, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    #conv2_test   = tf.nn.conv2d(conv2_in, conv2_W, strides=[1, 1, 1, 1], padding='VALID')
    conv2_2d   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    
    
    #conv2_test   = tf.nn.conv2d(conv2_in, conv2_W, strides=[1, 1, 1, 1], padding='VALID')
    conv2_re = sess.run(conv2_2d, feed_dict={x: X_test[0:1]})
    #conv2_re = sess.run(conv2_test, feed_dict={conv2_in: ones32x32x6})
    print("conv2: ", conv2_re.shape)    
    
    print("", file = flog)      
    for i in range(10):
        for j in range(10):
            for o in range(16):
                print("%d, "%(conv2_re[0][i][j][o]*256).astype(int), file = flog, end=' ')
                #print("%6d, "%(conv2_re[0][i][j][o]).astype(int), file = flog, end=' ')
            print("", file = flog)
        print("", file = flog)    
        
    pool2_re = sess.run(conv2, feed_dict={x: X_test[0:1]})
    print("==========================pool2_re==========================", file = flog)     
    print("pool2_re: ", pool2_re.shape)  
    print("", file = flog)      
    for i in range(5):
        for j in range(5):
            for o in range(16):
                print("%6d, "%(pool2_re[0][i][j][o]*256).astype(int), file = flog, end=' ')
            print("", file = flog)
        print("", file = flog)   
        
        
    fc0_re = sess.run(fc0, feed_dict={x: X_test[0:1]})
    print("fc0_re: ", fc0_re.shape)  
    print("==========================fc0_re==========================", file = flog)     
    
    for i in range(400):
        print("%6d, "%(fc0_re[0][i]*256).astype(int), file = flog, end=' ')
        if i%16==15 :
            print("", file = flog) 
    flog.close()      
        
        
        
####################################################        
    
    flog = open("test_digits.log", 'w')
    print("X_test: ", X_test.shape)
    for n in range(100):
        for i in range(32):
            for j in range(32):
                print("%4d, "%(X_test[n][i][j][0]*255).astype(int), file = flog, end='')
            print("", file = flog)
        print("", file = flog)
        print("", file = flog)
        
    fc3_re = sess.run(logits, feed_dict={x: X_test[0:100]})
    print("fc3_re: ", fc3_re.shape)
    print("==========================fc3_re==========================", file = flog)   
    
    for n in range(100):
        for i in range(10):
            print("%6d, "%(fc3_re[n][i]*256).astype(int), file = flog, end=' ')
        print("", file = flog)
        
        
    print("==========================digits==========================", file = flog)   
    y_test
    print("y_test: ", y_test.shape)
    for n in range(100):
        print("%d, %d"% (n, y_test[n]), file = flog, end=' ')
        print("", file = flog)  
        
    print("", file = flog)  
    
    flog.close()
    
    

INFO:tensorflow:Restoring parameters from .\lenet
conv1:  (1, 28, 28, 6)
relu1:  (?, 28, 28, 6)
pool1_re:  (1, 14, 14, 6)
conv2:  (1, 10, 10, 16)
pool2_re:  (1, 5, 5, 16)
fc0_re:  (1, 400)
X_test:  (10000, 32, 32, 1)
fc3_re:  (100, 10)
y_test:  (10000,)
